In [1]:
# Notebooks
import nbimporter
import os
import sys

# Functions from src
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
# Defined Functions
from utils import *

# Pandas, matplotlib, pickle, seaborn
import pickle
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from statistics import mean
from collections import Counter
from tgan.model import TGANModel

random_state = 13


/home/matias/anaconda3/envs/tgan/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/matias/anaconda3/envs/tgan/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/matias/anaconda3/envs/tgan/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/matias/anaconda3/envs/tgan/lib/python3.7/site-

/home/matias/anaconda3/envs/tgan/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/matias/anaconda3/envs/tgan/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/matias/anaconda3/envs/tgan/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/matias/anaconda3/envs/tga

In [2]:
# global variables/constants
num_trials = 30
test_size_percentage = 0.2 # for CV within train split
fixed_depth = 10


# Load Datasets (reduced labels)

## Occutherm

In [3]:
df_tcs_train = pd.read_pickle("data/occutherm/df_feature1_train_reduced.pkl") 
df_tcs_test = pd.read_pickle("data/occutherm/df_feature1_test_reduced.pkl")

# total count for instances per class: 818
print(df_tcs_train.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1508 entries, 0 to 1507
Data columns (total 10 columns):
Temperature (Fahrenheit)       1508 non-null float64
SkinTemperature                1508 non-null float64
ClothingInsulation             1508 non-null float64
Height(cm)                     1508 non-null float64
Shoulder Circumference(cm)     1508 non-null float64
Weight(lbs)                    1508 non-null float64
Gender                         1508 non-null int64
Temperature_outside            1508 non-null float64
Humidity_outside               1508 non-null float64
Discrete Thermal Comfort_TA    1508 non-null int64
dtypes: float64(8), int64(2)
memory usage: 117.9 KB
None


## Cresh (not required to be rerun for 3 labels since it already had that set up)

## ASHRAE

In [5]:
df_ashrae_train = pd.read_pickle("data/ashrae/ashrae_train_reduced.pkl")
df_ashrae_test = pd.read_pickle("data/ashrae/ashrae_test_reduced.pkl")

print(df_ashrae_train.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46477 entries, 0 to 46476
Data columns (total 7 columns):
SET                          46477 non-null float64
Clo                          46477 non-null float64
Met                          46477 non-null float64
Air temperature (¡C)         46477 non-null float64
Relative humidity (%)        46477 non-null float64
Air velocity (m/s)           46477 non-null float64
Thermal sensation rounded    46477 non-null float64
dtypes: float64(7)
memory usage: 2.5 MB
None


# Samplig synthetic datasets

https://sdv-dev.github.io/TGAN/

Update the `config.json` file according to the `TGAN` documentation and run `tgan_hyperparameters.sh` to find the best set of hyper-parameters for the model. 

Use these hyper-parameters in `train_tgan_<dataset>.sh` to train the model.

The generation and evaluation of generated samples is done in the next cells.


## Occutherm
Required samples

| Model | -1 | 0 | 1 |
|---|---|---|---|
| SMOTE | 342 | 0 | 603 |
| ADASYN | 247 | 0 | 621 |


## Cresh (not required to be rerun for 3 labels since it already had that set up)
Required samples

| Model | 9 | 10 | 11 |
|---|---|---|---|
| SMOTE | 547 | 0 | 433 |
| ADASYN | 534 | 0 | 402 |


## ASHRAE
Required samples

| Model  | -1 | 0 | 1 |
|--------|----|----|----|
| SMOTE  | 7686 | 0 | 3167 |
| ADASYN | 8141 | 0 | 3167 |


# TGAN


In [6]:
def sample_TGAN(df, dataset="occutherm", logs=False):
    # load require samples to balance and initial count
    if dataset == "occutherm":
        # samples to generate per label
        occutherm_req_one = 342
        occutherm_req_zero = 0 
        occutherm_req_minus_one = 603
        # count of generated samples per label
        count_one = 0
        count_minus_one = 0
    elif dataset == "cresh":
        # samples to generate per label
        cresh_req_eleven = 433
        cresh_req_ten = 0
        cresh_req_nine = 547
        # count of generated samples per label
        count_nine = 0
        count_eleven = 0
    elif dataset == "ashrae":
        # samples to generate per label
        ashrae_req_one = 7686
        ashrae_req_zero = 0
        ashrae_req_minus_one = 3167
        # count of generated samples per label
        count_one = 0
        count_minus_one = 0

    samples_count = 0 # total number of samples generated
    finish_loop = False
    
    # samples to generated every loop
    if dataset == "ashrae":
        print_threshold = 3000
    else:
        print_threshold = 2000

    # initiliaze synthetic dataframe
    columns = df.columns.values
    df_synth = pd.DataFrame(columns=columns)
    
    # load model
    model_path = "models/tgan_" + dataset + "_reduced" + ".pkl"
    tgan = TGANModel.load(model_path)

    while True:
        if finish_loop:
            samples_count += print_threshold
            break
        
        # generate `print_threshold` samples
        curr_df = tgan.sample(print_threshold)
        
        # classes generated
        print(curr_df.iloc[:, -1].unique()) # DEBUG
        
        # iterate through the generated samples
        if dataset == "occutherm":
            for index, row in curr_df.iterrows():  
                if (row['Discrete Thermal Comfort_TA'] == '1') and (count_one != occutherm_req_one): 
                    df_synth = df_synth.append(row)
                    count_one += 1
                elif (row['Discrete Thermal Comfort_TA'] == '-1') and (count_minus_one != occutherm_req_minus_one): 
                    df_synth = df_synth.append(row)
                    count_minus_one += 1
                elif (count_one == occutherm_req_one) and \
                    (count_minus_one == occutherm_req_minus_one):
                    finish_loop = True
                    break

        elif dataset == "cresh":
            for index, row in curr_df.iterrows():  
                if (row['thermal_cozie'] == '9') and (count_nine != cresh_req_nine): 
                    df_synth = df_synth.append(row)
                    count_nine += 1 
                elif (row['thermal_cozie'] == '11') and (count_eleven != cresh_req_eleven): 
                    df_synth = df_synth.append(row)
                    count_eleven += 1
                elif (count_nine == cresh_req_nine) and (count_eleven == cresh_req_eleven):
                    finish_loop = True
                    break

        elif dataset == "ashrae":
            for index, row in curr_df.iterrows(): 
                if (row['Thermal sensation rounded'] == '1.0') and (count_one != ashrae_req_one): 
                    df_synth = df_synth.append(row)
                    count_one += 1
                elif (row['Thermal sensation rounded'] == '-1.0') and (count_minus_one != ashrae_req_minus_one): 
                    df_synth = df_synth.append(row)
                    count_minus_one += 1
                elif (count_one == ashrae_req_one) and \
                     (count_minus_one == ashrae_req_minus_one):
                    finish_loop = True
                    break

        samples_count += print_threshold

    if logs:
        print('Original {} train dataset shape {}'.format(df, Counter(y)))        
        print('Resampled (TGAN) synth dataset shape %s' % Counter(np.array(df_synth.iloc[:, -1])))

    if dataset == "occutherm":
        df_synth['Gender'] = pd.to_numeric(df_synth['Gender'], errors='coerce')
    # last column in float instead of string
    df_synth.iloc[:, -1] = pd.to_numeric(df_synth.iloc[:, -1], errors='coerce')

    return df_synth


In [7]:
def evaluate_on_dataset(df_train, df_test, dataset='occutherm'):
    model = 'tgan'
    # empty list to hold values during trials    
    variability_list = []
    diversity_list = []

    class_acc_test_list_0 = []
    class_acc_test_list_1 = []
    class_acc_test_list_2 = []
    class_acc_test_list_3 = []
    
    class_report_rdf_list = []

    for i in range(0, num_trials):      
        ###################################
        # Sample synthethic dataset for 'dataset'
        df_synth = sample_TGAN(df_train, dataset) # complement needed for balancing dataset        
        # merge synthethic + real dataset
        df_real_synth = pd.concat([df_synth, df_train])
        
        ###################################
        # Variability of generated samples
        variability = evaluation_variability(df_synth)
        variability_list.append(variability)
    
        #################################################
        # Class diversity with respect to the training set
        diversity = evaluation_diversity(df_synth, df_train, baseline=False)
        diversity_list.append(diversity)

        #####################################
        # Quality on the final classification
        # use best models NB, KNN, SVM, RDF
        class_acc_test, class_acc_train, class_models, class_report_rdf = evaluation_classification(df_real_synth, df_test, rdf_depth=fixed_depth, depth_file_name='default', test_size_percentage=test_size_percentage)
        class_acc_test_list_0.append(class_acc_test[0])
        class_acc_test_list_1.append(class_acc_test[1])
        class_acc_test_list_2.append(class_acc_test[2])
        class_acc_test_list_3.append(class_acc_test[3])
        class_report_rdf_list.append(class_report_rdf)
        
        ########################
        # end of for loop trials
        print("End of {} trial".format(i + 1))
        
    # get average of trials
    variability = mean(variability_list)
    diversity = mean(diversity_list)
    class_acc_test = [mean(class_acc_test_list_0), mean(class_acc_test_list_1), mean(class_acc_test_list_2), mean(class_acc_test_list_3)]
    
    #####################################
    # Saving results
    # Format is folder/<dataset_string>-<experiment_name>_<metric or model>_<test or train>_<model>.pkl
    save_pickle(variability, "metrics/" + dataset + "-reduced" + "_variability_" + model + "_trials.pkl")
    save_pickle(diversity, "metrics/" + dataset + "-reduced" + "_diversity_" + model + "_trials.pkl")
    save_pickle(class_acc_test, "metrics/" + dataset + "-reduced" + "_classification_" + model + "_trials.pkl")
    save_pickle(class_report_rdf_list, "label-metrics/" + dataset + "-reduced" + "_class_report_" + model + "_trials.pkl")

    print("################################################################################")
    print("# Metrics and models for dataset {} saved!".format(dataset))
    print("################################################################################")


### Occutherm

In [8]:
evaluate_on_dataset(df_tcs_train, df_tcs_test, dataset='occutherm')









Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
[0710 13:44:16 @registry.py:126] gen/LSTM/00/FC input: [200, 300]

[0710 13:44:16 @registry.py:134] gen/LSTM/00/FC output: [200, 600]
[0710 13:44:16 @registry.py:126] gen/LSTM/00/FC2 input: [200, 600]
[0710 13:44:16 @registry.py:134] gen/LSTM/00/FC2 output: [200, 1]
[0710 13:44:16 @registry.py:126] gen/LSTM/01/FC input: [200, 300]
[0710 13:44:16 @registry.py:134] gen/LSTM/01/FC output: [200, 600]
[0710 13:44:16 @registry.py:126] gen/LSTM/01/FC2 input: [200, 600]
[0710 13:44:16 @registry.py:134] gen/LSTM/01/FC2 output: [200, 5]
[0710 13:44:16 @registry.py:126] gen/LSTM/01/FC3 input: [200, 5]
[0710 13:44:16 @registry.py:13

  4%|4         |9/200[00:00<00:14,12.77it/s]


['0' '-1' '1']


  4%|4         |9/200[00:00<00:03,55.75it/s]


['-1' '0' '1']
End of 1 trial
[0710 13:44:46 @collection.py:146] New collections created in tower : tf.GraphKeys.REGULARIZATION_LOSSES
[0710 13:44:46 @collection.py:165] These collections were modified but restored in : (tf.GraphKeys.SUMMARIES: 0->2)
[0710 13:44:46 @sessinit.py:87] WRN The following variables are in the checkpoint, but not found in the graph: global_step, optimize/beta1_power, optimize/beta2_power
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
[0710 13:44:47 @sessinit.py:114] Restoring checkpoint from output/model/model-50000 ...
INFO:tensorflow:Restoring parameters from output/model/model-50000


...

['0' '-1' '1']
End of 29 trial
[0710 13:54:10 @collection.py:146] New collections created in tower : tf.GraphKeys.REGULARIZATION_LOSSES
[0710 13:54:10 @collection.py:165] These collections were modified but restored in : (tf.GraphKeys.SUMMARIES: 0->2)
[0710 13:54:10 @sessinit.py:87] WRN The following variables are in the checkpoint, but not found in the graph: global_step, optimize/beta1_power, optimize/beta2_power
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
[0710 13:54:11 @sessinit.py:114] Restoring checkpoint from output/model/model-50000 ...
INFO:tensorflow:Restoring parameters from output/model/model-50000


  4%|4         |9/200[00:00<00:10,18.70it/s]


['0' '-1' '1']


  4%|4         |9/200[00:00<00:03,62.68it/s]


['-1' '1' '0']
End of 30 trial
################################################################################
# Metrics and models for dataset occutherm saved!
################################################################################


### Cresh (not required to be rerun for 3 labels since it already had that set up)

## ASHRAE


In [10]:
evaluate_on_dataset(df_ashrae_train, df_ashrae_test, dataset='ashrae')


[0710 16:05:22 @collection.py:146] New collections created in tower : tf.GraphKeys.REGULARIZATION_LOSSES
[0710 16:05:22 @collection.py:165] These collections were modified but restored in : (tf.GraphKeys.SUMMARIES: 0->2)
[0710 16:05:22 @sessinit.py:87] WRN The following variables are in the checkpoint, but not found in the graph: global_step, optimize/beta1_power, optimize/beta2_power
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
[0710 16:05:23 @sessinit.py:114] Restoring checkpoint from output/model/model-50000 ...
INFO:tensorflow:Restoring parameters from output/model/model-50000


  7%|7         |14/200[00:00<00:05,33.85it/s]


['0.0' '1.0' '-1.0']


  7%|7         |14/200[00:00<00:01,103.36it/s]


['0.0' '-1.0' '1.0']


...

['0.0' '1.0' '-1.0']


  7%|7         |14/200[00:00<00:01,120.68it/s]


['0.0' '-1.0' '1.0']


  7%|7         |14/200[00:00<00:01,123.68it/s]


['0.0' '1.0' '-1.0']
End of 30 trial
################################################################################
# Metrics and models for dataset ashrae saved!
################################################################################
